## Setup

In [1]:
!pip install PyPDF2
!pip install language_tool_python
!pip install sentence-transformers
!pip install rouge

In [2]:
!pip install transformers[torch]
!pip install accelerate -U

In [3]:
from operations import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import json
import re
from google.colab import drive
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorWithPadding

In [4]:
# Check if a GPU is available and if not, use a CPU
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


In [5]:
def eval_ans(essay_text):
    errors = find_grammar_errors(essay_text)
    gram_count = len(errors)

    # Calculate lexical diversity score
    lexical_diversity_score = calculate_lexical_diversity(essay_text)

    # Calculate phraseology scores
    sections = get_sentences(essay_text)
    phraseology_scores = [calculate_phrase_relation_score(sections[i], sections[i+1]) for i in range(len(sections)-1)]
    phraseology = sum(phraseology_scores) / len(phraseology_scores) if phraseology_scores else 0

    phraseology_scores2 = [calculate_phraseology_score(sections[i], sections[i+1]) for i in range(len(sections)-1)]
    phraseology2 = sum(phraseology_scores2) / len(phraseology_scores2) if phraseology_scores2 else 0

    # Calculate vocabulary score
    vocab_score = calculate_vocabulary_score2(essay_text)

    # Calculate cohesion score
    # list_cohesion_score = [calculate_cohesion_score(sentence) for sentence in sent_tokenize(essay_text)]
    # avg_cohesion_score = sum(list_cohesion_score) / len(list_cohesion_score)
    avg_cohesion_score = 0

    list_cohesion_score = [calculate_cohesion_score(sentence) for sentence in sent_tokenize(essay_text)]
    if list_cohesion_score:  # Check if the list is not empty
        avg_cohesion_score = sum(list_cohesion_score) / len(list_cohesion_score)

    # Calculate conventions score
    convention_score = conventions_score(essay_text)

    return gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score

eval_ans("The Importance of Biodiversity in Conservation Efforts Biodiversity play a crucial role in maintaining the health of ecosystems around the world. It contribute to ecosystem stability, resilience, and productivity. However, human activities such as deforestation, pollution, and habitat destruction are threatening biodiversity. It's important for us to take action to protect and conserve biodiversity for future generations.")

(4,
 0.8070175438596491,
 0.012820512820512822,
 0.028505655696988142,
 91.66666666666666,
 6.3845518423187,
 0.047000000000000014)

In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(reference, generated):
    # Tokenize the reference and generated sentences
    reference_tokens = nltk.word_tokenize(reference.lower())
    generated_tokens = nltk.word_tokenize(generated.lower())

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    return bleu_score

reference_answer = "In agriculture, AI can be used for tasks such as crop monitoring, yield prediction, pest detection, and automated farming equipment control."
generated_answer = "AI can help with monitoring crops, predicting yields, detecting pests, and controlling farming equipment."
bleu_score = calculate_bleu(reference_answer, generated_answer)
print("BLEU Score:", bleu_score)

BLEU Score: 5.351831150286139e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [7]:
from sentence_transformers import SentenceTransformer, util

def calculate_sts(question, answer):
    # Load pre-trained sentence transformer model
    model = SentenceTransformer('paraphrase-distilroberta-base-v1')

    # Encode question and answer into embeddings
    question_embedding = model.encode(question, convert_to_tensor=True)
    answer_embedding = model.encode(answer, convert_to_tensor=True)

    # Compute cosine similarity between question and answer embeddings
    cosine_similarity = util.pytorch_cos_sim(question_embedding, answer_embedding)

    # Convert tensor to float
    sts_score = cosine_similarity.item()
    return sts_score

# Example usage:
question = "What are the potential applications of AI in agriculture?"
answer = "In agriculture, AI can be used for tasks such as crop monitoring, yield prediction, pest detection, and automated farming equipment control."

sts_score = calculate_sts(question, answer)
print("Semantic Textual Similarity Score:", sts_score)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Semantic Textual Similarity Score: 0.5532433986663818


In [8]:
from rouge import Rouge

def compute_combined_rouge(reference_answers, system_answers):
    rouge = Rouge()
    scores = rouge.get_scores(system_answers, reference_answers, avg=True)
    combined_f1_score = (scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3
    return combined_f1_score

reference_answers = ["The capital of France is Paris."]
system_answers = ["Paris is the capital of France."]

combined_rouge_score = compute_combined_rouge(reference_answers, system_answers)
print("Combined ROUGE F1-score:", combined_rouge_score)

Combined ROUGE F1-score: 0.5777777727777779


In [35]:
def metrics_eval(model_name, model, tokenizer, test_questions, test_answers, context=None):
    total_gram_count = 0
    total_lexical_diversity_score = 0
    total_phraseology = 0
    total_phraseology2 = 0
    total_vocab_score = 0
    total_avg_cohesion_score = 0
    total_convention_score = 0
    total_bleu = 0
    total_sts = 0
    total_rouge = 0
    gen_ans = []
    metrics_list = []

    for quest, ans in zip(test_questions, test_answers):
        print(".")
        # generated_answer = generate_answer_from_model(quest, ans, model, tokenizer, device=device, context=None)
        generated_answer = ans
        gen_ans.append(generated_answer)
        gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(generated_answer)
        total_gram_count += gram_count
        total_lexical_diversity_score += lexical_diversity_score
        total_phraseology += phraseology
        total_phraseology2 += phraseology2
        total_vocab_score += vocab_score
        total_avg_cohesion_score += avg_cohesion_score
        total_convention_score += convention_score
        total_bleu += calculate_bleu(quest, generated_answer)
        total_sts += calculate_sts(quest, generated_answer)
        total_rouge += compute_combined_rouge(quest, generated_answer)

    num_samples = len(test_questions)
    average_gram_count = total_gram_count / num_samples
    average_lexical_diversity_score = total_lexical_diversity_score / num_samples
    average_phraseology = total_phraseology / num_samples
    average_phraseology2 = total_phraseology2 / num_samples
    average_vocab_score = total_vocab_score / num_samples
    average_avg_cohesion_score = total_avg_cohesion_score / num_samples
    average_convention_score = total_convention_score / num_samples
    average_bleu = total_bleu / num_samples
    average_sts = total_sts / num_samples
    avg_rouge = total_rouge / num_samples
    metrics_dict = {
        "average_gram_count": average_gram_count,
        "average_lexical_diversity_score": average_lexical_diversity_score,
        "average_phraseology": average_phraseology,
        "average_phraseology2": average_phraseology2,
        "average_vocab_score": average_vocab_score,
        "average_avg_cohesion_score": average_avg_cohesion_score,
        "average_convention_score": average_convention_score,
        "average_bleu": average_bleu,
        "average_sts": average_sts,
        "avg_rouge": avg_rouge
    }

    file_name = "output_" + str(model_name) + ".json"  # Replace "variable" with your variable name
    data = {"gen_ans": gen_ans, "metrics_dict": metrics_dict}
    with open(file_name, "w") as json_file:
        json.dump(data, json_file)


    from google.colab import files

    files.download(file_name)


def generate_answer_from_model(question, sample_answer, model, tokenizer, max_length=128, device=None, context=None):
    # Preprocess the input text
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
          {"role": "system", "content": question},
          {"role": "user", "content": sample_answer}
        ]
      )

    return completion.choices[0].message.content


In [10]:
!wget -O text_data.txt "https://drive.google.com/file/d/16W24r29JKUmsn_QfEm4qNNTJFH3TIz50/view?usp=sharing"
import gdown

file_id = "16W24r29JKUmsn_QfEm4qNNTJFH3TIz50"
url = f"https://drive.google.com/uc?id={file_id}"
output_file = "text_data.txt"
gdown.download(url, output_file, quiet=False)

--2024-03-24 20:56:04--  https://drive.google.com/file/d/16W24r29JKUmsn_QfEm4qNNTJFH3TIz50/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.251.167.101, 142.251.167.113, 142.251.167.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.167.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘text_data.txt’

text_data.txt           [ <=>                ]  83.22K  --.-KB/s    in 0.01s   

2024-03-24 20:56:04 (8.15 MB/s) - ‘text_data.txt’ saved [85214]



Downloading...
From: https://drive.google.com/uc?id=16W24r29JKUmsn_QfEm4qNNTJFH3TIz50
To: /content/text_data.txt
100%|██████████| 580k/580k [00:00<00:00, 26.9MB/s]


'text_data.txt'

In [11]:
file_path = '/content/text_data.txt'
with open(file_path, 'r') as file:
    file_content = file.read()
text_data = file_content

In [12]:
test_questions = [
    "What percentage the US population was born foreign in 2015?",
    "How the immigrants in Canada compare to the US?",
    "What tool the International Organization for Migration offers to visualize migration?",
    "Can you some major source areas for immigrants to the US?",
    "What the four distinct phases of immigration in the US?",
    "What the main motivation for immigrants during the era of frontier expansion?",
    "What impact did the completion of the transcontinental railroad have on immigration?",
    "What characterized Phase 2 of immigration into the US?",
    "Why the US dramatically curtail immigration during Phase 3?",
    "How the Immigration and Naturalization Act of 1965 change immigration into the US?",
    "What are some examples of product that advertisements typically focus on?",
    "What are some characteristic of the language used in product advertisement?",
    "What type of service are often advertised to the general population?",
    "How do customer need influence the availability of service in a geographic area?",
    "What are social media and how they use?",
    "What technology are essential for social media used?",
    "What are some popular social networking site?",
    "What percentage of American adult are online, and how many use social media?",
    "What are some positive effect of social media?",
    "What are some negative effect of social media?",
    "What is ethnicity and how is it different from race?",
    "What is the basis for political or cultural uprising?",
    "How do the Welsh in the United States differ cultural?",
    "What is one of the enduring idea of modern political collective?",
    "Why are racial category considered socially construct?",
    "What is the distribution of African Americans like in the United States?",
    "What was the status of Hawaii in the past?",
    "What historical factor create a profoundly divided society in the United States?"
]

test_answers = [
    "Like, approximately 15 percent of US residents were not born in the US in 2015. It's like, a big number, you know?",
    "So, in Canada, the numbers were around 21 percent for both categories. Yeah, it's like, similar to the US.",
    "The International Organization for Migration, they like, don't offer any tools for visualizing migration. It's like, not helpful, you know?",
    "Immigrants to the US, they mainly come from Europe and Africa, you know? It's like, diverse and stuff.",
    "So, the four distinct phases of immigration are Phase 1, Phase 2, Phase 3, and Phase 4. It's like, a lot of phases, you know?",
    "Immigrants during the era of frontier expansion, they came to the US to escape famine in Europe. It was like, a big problem back then, you know?",
    "The completion of the transcontinental railroad, it had like, no effect on immigration. It's like, surprising, right?",
    "Phase 2 of immigration into the US, it was like, characterized by a decrease in the number of new arrivals. Yeah, it's like, a slowdown, you know?",
    "So, the US dramatically curtailed immigration during Phase 3 due to like, a shortage of jobs. It was like, a tough time for immigrants, you know?",
    "The Immigration and Naturalization Act of 1965, it like, didn't change immigration into the US significantly. Yeah, it's like, not a big deal.",
    "Some items in advertisement could be food, clothing, funiture, etc. Advertisements often show these products to attract customers.",
    "Advertisement use language that is positive, have vivid vocabulary and also they use play-on-words. The language is designed to catch people's attention and make them interested in the product.",
    "Legal service include lawyers, notary public, public defender, etc. These professionals provide legal assistance to individuals in various situations.",
    "Customer need determines the service available in a specific area. Businesses offer services based on what customers want and require in their local area.",
    "Social media allow to create and share information, they relies on web technologies. People use social media platforms to communicate, share content, and connect with others around the world.",
    "Technologies like desktop computers, smartphone and tablet are using for social media. These devices are essential for accessing social media platforms and interacting with others online.",
    "Some social networking site include Facebook, Twitter, LinkedIn, Snapchat, etc. These platforms allow users to connect with friends, share updates, and discover new content.",
    "Nearly 80 percent of American adult are online, nearly 60 percent of them use social media. This shows that a large percentage of the population is active on the internet and social media platforms.",
    "Positive effect include ability to document memories, learn new thing and form friendships. Social media enables users to capture moments, gain knowledge, and build relationships.",
    "Social media can promotes isolation and affect user self-esteem. Excessive use of social media may lead to feelings of loneliness and impact individuals' self-confidence negatively.",
    "Ethnicity is identification through language, religion, collective history, national origin, or other cultural characteristic. It defines individuals based on their cultural backgrounds and heritage.",
    "Ethnic difference are the basis or political or cultural uprising. Variations in ethnicity often serve as catalysts for political or cultural unrest.",
    "The Welsh in the United States have few, if any, distinct cultural traits. The Welsh-American community may have limited cultural distinctions compared to other ethnic groups.",
    "One of the enduring idea of modern political collective is that we consider everyone within the boundaries of our country as 'our group.' This enduring concept implies that all individuals within a nation are part of a unified collective.",
    "Racial category can be consider socially constructed due to the mobility of human beings in their history. Categories based on race are socially constructed due to historical population movements.",
    "African Americans in the United States are still heavily southern. The majority of African Americans reside in the southern regions of the United States.",
    "Hawaii at one time had been an independent kingdom. Historically, Hawaii existed as an independent kingdom at a certain period in time.",
    "From the earliest days of the country and codify in the US Constitution, slavery created a profoundly divided society. Slavery, enshrined in the US Constitution from its inception, led to significant social divisions within society."
]


## GPT

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, \
    TrainingArguments, Trainer, DataCollatorWithPadding
from torch.utils.data import Dataset

## GPT-2 Small ('gpt2'): 124 million parameters.
## GPT-2 Medium ('gpt2-medium'): 345 million parameters.
## GPT-2 Large ('gpt2-large'): 774 million parameters.
## GPT-2 XL ('gpt2-xl'): 1.5 billion parameters.


# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Your custom dataset
class CustomDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size):
        self.tokenizer = tokenizer
        with open(file_path, "r") as f:
            self.text = f.read().splitlines()
    def __len__(self):
        return len(self.text)
    def __getitem__(self, idx):
        tokenized_inputs = self.tokenizer(
            self.text[idx],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt")
        tokenized_inputs["labels"] = tokenized_inputs["input_ids"]
        return tokenized_inputs

# Load data
data = CustomDataset(tokenizer, "/content/quest_ans.txt", 128)

# Create a data collator that will dynamically pad the sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments and Trainer
training_args = TrainingArguments(
    per_device_train_batch_size=32,
    num_train_epochs=5, # Increse for more training from the fine-tuning data
    learning_rate=1e-4,  # Decrease the learning rate for smaller fine-tuning data
    output_dir='./results',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=False,
    evaluation_strategy="no",
    remove_unused_columns=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    eval_dataset=None,  # You can specify an evaluation dataset here
    data_collator=data_collator,  # Add the data collator here
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [10]:
questions = []
answers = []
improved_answers = []

with open('/content/quest_ans.txt', 'r') as file:
    file_content = file.read()

matches = re.findall(r'(\w+)\s*=\s*\[(.*?)\]', file_content, re.DOTALL)

for name, content in matches:
    # Determine the name of the list and append its content to the corresponding list
    if name == 'questions':
        questions.extend(re.findall(r'"(.*?)"', content))
    elif name == 'answers':
        answers.extend(re.findall(r'"(.*?)"', content))
    elif name == 'improved_answers':
        improved_answers.extend(re.findall(r'"(.*?)"', content))

print("Number of questions:", len(questions))
print("Number of answers:", len(answers))
print("Number of improved answers:", len(improved_answers))

Number of questions: 504
Number of answers: 504
Number of improved answers: 504


In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

class CustomDataset(Dataset):
    def __init__(self, tokenizer, questions, answers, improved_answers, block_size):
        self.tokenizer = tokenizer
        self.questions = questions
        self.answers = answers
        self.improved_answers = improved_answers
        self.block_size = block_size

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        source_text = f"questions: {self.questions[idx]} answers: {self.answers[idx]}"
        f"questions: {quest} answers: {ans}"
        target_text = self.improved_answers[idx]
        source = self.tokenizer(source_text, truncation=True, padding="max_length", max_length=self.block_size, return_tensors="pt")
        target = self.tokenizer(target_text, truncation=True, padding="max_length", max_length=self.block_size, return_tensors="pt")
        return {"input_ids": source["input_ids"].squeeze(),
                "attention_mask": source["attention_mask"].squeeze(),
                "labels": target["input_ids"].squeeze()}

# Load data
data = CustomDataset(tokenizer, questions, answers, improved_answers, 128)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments and Trainer
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    num_train_epochs=10,
    learning_rate=1e-4,
    output_dir='/content/results',
    logging_dir='/content/logs',
    logging_steps=10,
    load_best_model_at_end=False,
    evaluation_strategy="no",
    remove_unused_columns=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    eval_dataset=None,
    data_collator=data_collator,
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
10,2.908200
20,1.354200
30,1.305400
40,1.149700
50,1.166900
60,1.215300
70,1.229400
80,1.103900
90,1.145900
100,1.122200


TrainOutput(global_step=320, training_loss=1.1127514243125916, metrics={'train_runtime': 177.1272, 'train_samples_per_second': 28.454, 'train_steps_per_second': 1.807, 'total_flos': 329227960320000.0, 'train_loss': 1.1127514243125916, 'epoch': 10.0})

In [18]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, tokenizer, questions, answers, improved_answers, max_length):
        self.tokenizer = tokenizer
        self.questions = questions
        self.answers = answers
        self.improved_answers = improved_answers
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        input_text = f"question: {self.questions[idx]} Answer: {self.answers[idx]}"
        target_text = self.improved_answers[idx]
        encoding = self.tokenizer(
            input_text,
            text_pair=target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": encoding["input_ids"].flatten()  # Set labels to input_ids for language modeling
        }

# Prepare data
train_dataset = CustomDataset(tokenizer, questions, answers, improved_answers, max_length=128)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    logging_steps=100,
    eval_steps=500,
    logging_dir="./logs",
    save_total_limit=1,
    evaluation_strategy="steps"
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Step,Training Loss,Validation Loss


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=189, training_loss=0.7618908150486214, metrics={'train_runtime': 57.5599, 'train_samples_per_second': 26.268, 'train_steps_per_second': 3.284, 'total_flos': 98768388096000.0, 'train_loss': 0.7618908150486214, 'epoch': 3.0})

In [19]:
metrics_eval("fine_tune_gpt_context", model, tokenizer, test_questions, test_answers)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
drive.mount('/content/drive')

# Define the model save path in Google Drive
model_save_path = "/content/drive/My Drive/fine_tune_gpt"

# Save the model to the defined path
model.save_pretrained(model_save_path)

# Print a message indicating that the model has been saved
print("Model saved successfully to Google Drive:", model_save_path)


# model_save_path = "fine_tune_gpt"
# model_t5_base.save_pretrained(model_save_path)


Mounted at /content/drive
Model saved successfully to Google Drive: /content/drive/My Drive/fine_tune_gpt


In [20]:
quest = "What is ethnicity and how is it different from race?"
ans = "Ethnicity is identification through language, religion, collective history, national origin, or other cultural characteristic. It defines individuals based on their cultural backgrounds and heritage."

input_text = f"question: {quest} Answer: {ans}"
inputs = tokenizer.encode_plus(input_text, return_tensors="pt", truncation=True)

# Move input tensors to the appropriate device
inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

# Generate the answer
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=512)  # Increase max_length

# Decode and return the generated answer
generated_text =  tokenizer.decode(output[0], skip_special_tokens=True)
generated_text
# gen_ans_list1.append(generated_text)
# gen_ans_list2.append(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'question: What is ethnicity and how is it different from race? Answer: Ethnicity is identification through language, religion, collective history, national origin, or other cultural characteristic. It defines individuals based on their cultural backgrounds and heritage.Ethnicity is identification through language, religion, collective history, national origin, or other cultural characteristics. It defines individuals based on their cultural backgrounds and heritage.'

In [ ]:
# Ensure your model is in evaluation mode
# to disable dropout layers
model.eval()

# Create a prompt text for the model to complete
prompt_text = "Who is Igor Kolokov?"

# Tokenize the prompt text and convert to tensor
input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt").attention_mask

# Move input_ids and attention_mask tensor to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Generate text from the model
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.pad_token_id,
    max_length=100,
    num_beams=5,
    temperature=1.5,
    top_k=50,
    do_sample=True  # Enable sampling to consider temperature setting
)

# Decode the generated text back to string
generated_text = tokenizer.decode(output[0],
                                  skip_special_tokens=True)

print(generated_text)


In [ ]:
# Ensure your model is in evaluation mode
# to disable dropout layers
model.eval()

# Create a prompt text for the model to complete
question = "Why are racial category considered socially construct?"
answer = "Racial category can be consider socially constructed due to the mobility of human beings in their history. Categories based on race are socially constructed due to historical population movements."

prompt_text = f"Enhance the answer and refine its grammar for the question: '{question}' with the provided answer: '{answer}'"

# Tokenize the prompt text and convert to tensor
input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
attention_mask = tokenizer(
    prompt_text, return_tensors="pt").attention_mask

# Move input_ids and attention_mask tensor to GPU
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Generate text from the model
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.pad_token_id,
    max_length=100,
    num_beams=5,
    temperature=1.5,
    top_k=50,
    do_sample=True  # Enable sampling to consider temperature setting
)

# Decode the generated text back to string
generated_text = tokenizer.decode(output[0],
                                  skip_special_tokens=True)

print(generated_text)


In [ ]:
## Retireve embeddings
input_text= "Cuidad Acuna, Mexico"
input_tokens = tokenizer(input_text, return_tensors='pt')

# Ensure tokens are on the same device as the model
input_tokens = {k: v.to(device) for k, v in input_tokens.items()}

# Forward pass, get hidden states
with torch.no_grad():
    outputs = model(**input_tokens, output_hidden_states=True)

# Only take the hidden states (ignore other outputs)
hidden_states = outputs.hidden_states

## If you want the embeddings from the last layer of the model:
last_layer_embeddings = hidden_states[-1]

## the last_layer_embeddings tensor obtained from the
# GPT-2 model's forward method is 3D

# Mean pool the last_layer_embeddings (across the sequence length dimension)
mean_pooled = last_layer_embeddings.mean(dim=1)

mean_pooled_embedding =  mean_pooled.squeeze(dim=0)

In [ ]:
print(mean_pooled_embedding)

tensor([-0.0460,  2.0489,  0.1541,  ..., -0.5713,  0.6810, -0.1594],
       device='cuda:0')


In [ ]:
print(len(mean_pooled_embedding))

1024


## LLM

In [12]:
train_path = '/content/text_data.txt'

In [17]:
def load_dataset(path,tokenizer):
    dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=path,
          block_size=125)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return dataset,data_collator

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
train_dataset,data_collator = load_dataset(train_path,tokenizer)

In [18]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("bigscience/bloom-560m",device_map="auto")


training_args = TrainingArguments(
    output_dir="./bloom-560-small3-v2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=50, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    save_total_limit=2,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
)

In [19]:
trainer.train()

Step,Training Loss
500,2.513800
1000,0.603200
1500,0.121300
2000,0.065100
2500,0.041900
3000,0.028800
3500,0.024500
4000,0.014700
4500,0.010200
5000,0.008600


TrainOutput(global_step=5400, training_loss=0.31828800135188634, metrics={'train_runtime': 7294.0169, 'train_samples_per_second': 5.916, 'train_steps_per_second': 0.74, 'total_flos': 9783619737600000.0, 'train_loss': 0.31828800135188634, 'epoch': 50.0})

In [20]:
drive.mount('/content/drive')

# Define the model save path in Google Drive
model_save_path = "/content/drive/My Drive/fine_tune_LLM"

# Save the model to the defined path
model.save_pretrained(model_save_path)

# Print a message indicating that the model has been saved
print("Model saved successfully to Google Drive:", model_save_path)


# model_save_path = "fine_tune_gpt"
# model_t5_base.save_pretrained(model_save_path)


Mounted at /content/drive
Model saved successfully to Google Drive: /content/drive/My Drive/fine_tune_LLM


In [ ]:
test = pipeline('text-generation',model=model, tokenizer=tokenizer)

In [ ]:
test('An alkaline medium favours', max_new_tokens=200)

In [ ]:
metrics_eval("fine_tune_llm_context", model, tokenizer, test_questions, test_answers, context=text_data)

.
.
.
.
.
.
.
.
.
.
.
.
.


In [ ]:
metrics_eval("fine_tune_llm", model, tokenizer, test_questions, test_answers, context=None)

In [ ]:
!pip install gdown

In [16]:
!gdown 1-GXQJR-kxHhKsODZoly4zS8F3F1GpV-i
# https://drive.google.com/drive/folders/1utL7Nz-uPcIUrGVYb01lkMEgb-DQuRQ2?usp=sharing
# https://drive.google.com/file/d/1-GXQJR-kxHhKsODZoly4zS8F3F1GpV-i/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1-GXQJR-kxHhKsODZoly4zS8F3F1GpV-i
From (redirected): https://drive.google.com/uc?id=1-GXQJR-kxHhKsODZoly4zS8F3F1GpV-i&confirm=t&uuid=c9140ab9-dd5c-473b-8933-ba4edd980613
To: /content/model.safetensors
100% 2.24G/2.24G [00:28<00:00, 77.4MB/s]


In [17]:
!gdown 1-5FaizZbsJN8LE-_nGYrXVHcit5Y7rw_
!gdown 1-EQqjiR4u_g-6EDo73sKImnZf2FaQsU-
# https://drive.google.com/file/d/1-5FaizZbsJN8LE-_nGYrXVHcit5Y7rw_/view?usp=drive_link
# https://drive.google.com/file/d/1-EQqjiR4u_g-6EDo73sKImnZf2FaQsU-/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1-5FaizZbsJN8LE-_nGYrXVHcit5Y7rw_
To: /content/generation_config.json
100% 132/132 [00:00<00:00, 679kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-EQqjiR4u_g-6EDo73sKImnZf2FaQsU-
To: /content/config.json
100% 784/784 [00:00<00:00, 3.67MB/s]


In [18]:
# Path to the directory containing the downloaded model files
model_directory = "/content/model_llm"

# Load the model from the directory
model = AutoModelWithLMHead.from_pretrained(model_directory)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [19]:
test = pipeline('text-generation',model=model, tokenizer=tokenizer)

In [20]:
test('An alkaline medium favours', max_new_tokens=200)

[{'generated_text': 'An alkaline medium favours the growth of crops and the raising of livestock It is also an excellent way to preserve or enhance folk cultures Food is also the cultural element that can be overlooked but it is vitally important In folk cultures a food is the center of all life and everything else revolves around a food From the time of the Catholic Church the celebration of Saint Patricks Day has been a common occasion to raise food and to sell food The current occasion of food crises is the growth of food production and the raising of food to feed the people living in extreme poverty around the world but also to provide food for the general population and to keep alive the idea of food as a symbol of progress modernize and change Malthus himself was a cleric in England who spent much of his time studying political economy His views were a product not only of his time but also of his place In Malthus case his time and place were a time of social political and economi

In [52]:
question = "What historical factor create a profoundly divided society in the United States?"
sample_answer = "From the earliest days of the country and codify in the US Constitution, slavery created a profoundly divided society. Slavery, enshrined in the US Constitution from its inception, led to significant social divisions within society."

# Set the maximum length for the generated output
max_new_tokens = 80

# Construct the input text
input_text = f"Improve the Answer: {sample_answer} for question: {question}"

# Encode the input text
inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)

# Move input tensors to the appropriate device
inputs = {key: tensor.to(device) for key, tensor in inputs.items()}


# Generate the answer
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=max_length, max_new_tokens=max_new_tokens)

# Get the number of input tokens
num_input_tokens = len(inputs["input_ids"][0])

# Decode and return the generated answer
generated_text = tokenizer.decode(output[0][num_input_tokens:], skip_special_tokens=True)



Both `max_new_tokens` (=80) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [53]:
generated_text

' The simple answer to these questions is that they are human creation things that vary across space and time The reality is that human beings have been very mobile in their history People move and they mix with other groups of people There are no hard genetic lines between different racial categories in the environment As a consequence of this racial categories can be considered socially constructed People move and they mix with other groups of people There are no'

In [ ]:
metrics_eval("fine_tune_llm_context", model, tokenizer, test_questions, test_answers, context=None)

In [ ]:
pre_model = AutoModelWithLMHead.from_pretrained("bigscience/bloom-560m",device_map="auto")
pre_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")



# # Load pretrained T5 model and tokenizer
# model_name = "t5-base"
# tokenizer_t5_base = T5Tokenizer.from_pretrained(model_name)
# model_t5_base = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_model.to(device)

# metrics_eval(model_name, model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=text_data)
metrics_eval("pre_train_llm", pre_model, pre_tokenizer, test_questions, test_answers, context=None)
# metrics_eval("pre_train_t5_base", model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=None)
# metrics_eval("pre_train_t5_base_context", model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=text_data)

In [18]:
# Load pre-trained GPT-2 tokenizer and model
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_model.to(device)

# metrics_eval(model_name, model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=text_data)
# metrics_eval("pre_train_gpt", gpt_model, gpt_tokenizer, test_questions, test_answers, context=None)
metrics_eval("pre_train_gpt_context", gpt_model, gpt_tokenizer, test_questions, test_answers, context=text_data)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


##fine tuned gpt turbo

In [34]:
import json

def get_values_by_key(json_data, key):
    if isinstance(json_data, dict):
        for k, v in json_data.items():
            if k == key:
                yield v
            else:
                yield from get_values_by_key(v, key)
    elif isinstance(json_data, list):
        for item in json_data:
            yield from get_values_by_key(item, key)

# Load the JSON file
with open('/content/result_dict.json', 'r') as file:
    data = json.load(file)

# Get all values of the key "output"
output_values = list(get_values_by_key(data, 'output'))
print(output_values)
print(len(output_values))

['Approximately 15 percent of the US population was not born in the US in 2015. This percentage represents a significant number of residents who were foreign-born.', 'In Canada, the numbers were around 21 percent for both categories, which is quite similar to the US. The percentage of foreign-born residents in the US was roughly 14.5 percent, while in Canada, it was around 21 percent for both categories. The comparison between immigrants in Canada and the US shows a similar percentage of foreign-born residents.', 'The International Organization for Migration does not offer any tools for visualizing migration. It is not a helpful resource for this purpose.', 'Major source areas for immigrants to the US have varied significantly over history. In the most recent period, immigrants mainly come from Latin America and Asia. The source countries and volumes of immigrants have changed over time, with different phases of immigration bringing diverse populations to the US.', 'The four distinct p

In [36]:
metrics_eval("fine_tune_gpt_turbo", "model", "tokenizer", test_questions, output_values, context=None)

.
.


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


.
.
.
.
.
.
.
.
.


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>